# minimal example of ontram implementation
- anlogous to https://github.com/liherz/ontram_pytorch.git

In [1]:
# Load dependencies
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split



from utils.graph import *
from utils.loss_ordinal import *
from utils.tram_model_helpers import *
from utils.tram_models import *
from utils.tram_data import *


import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Train with GPU support.")
else:
    device = torch.device('cpu')
    print("No GPU found, train with CPU support.")

Train with GPU support.


adjustet funcitnos for ordinal outcomes 

dev ordinal

In [2]:
experiment_name = "ordinal_wine_example"   ## <--- set experiment name
seed=42
np.random.seed(seed)

LOG_DIR="/home/bule/TramDag/dev_experiment_logs"
EXPERIMENT_DIR = os.path.join(LOG_DIR, experiment_name)
DATA_PATH = EXPERIMENT_DIR # <----------- change to different source if needed
CONF_DICT_PATH = os.path.join(EXPERIMENT_DIR, f"configuration.json")

In [3]:
# Load the dataset
wine = load_wine()
wine

{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [4]:
df=pd.DataFrame(wine['data'], columns=wine['feature_names'])
df['target']=wine['target']

In [5]:
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

quantiles = train_df.quantile([0.05, 0.95])
min_vals = quantiles.loc[0.05]
max_vals = quantiles.loc[0.95]

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 142 entries, 158 to 102
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       142 non-null    float64
 1   malic_acid                    142 non-null    float64
 2   ash                           142 non-null    float64
 3   alcalinity_of_ash             142 non-null    float64
 4   magnesium                     142 non-null    float64
 5   total_phenols                 142 non-null    float64
 6   flavanoids                    142 non-null    float64
 7   nonflavanoid_phenols          142 non-null    float64
 8   proanthocyanins               142 non-null    float64
 9   color_intensity               142 non-null    float64
 10  hue                           142 non-null    float64
 11  od280/od315_of_diluted_wines  142 non-null    float64
 12  proline                       142 non-null    float64
 13  target  

In [11]:
train_df['target']

158    2
137    2
98     1
159    2
38     0
      ..
71     1
106    1
14     0
92     1
102    1
Name: target, Length: 142, dtype: int64

data types

In [7]:
data_type={key:value for key, value in zip(train_df.columns, ['cont']*13+['ord'])}
data_type

{'alcohol': 'cont',
 'malic_acid': 'cont',
 'ash': 'cont',
 'alcalinity_of_ash': 'cont',
 'magnesium': 'cont',
 'total_phenols': 'cont',
 'flavanoids': 'cont',
 'nonflavanoid_phenols': 'cont',
 'proanthocyanins': 'cont',
 'color_intensity': 'cont',
 'hue': 'cont',
 'od280/od315_of_diluted_wines': 'cont',
 'proline': 'cont',
 'target': 'ord'}

configartion dicitonary

In [8]:
configuration_dict=new_conf_dict(experiment_name,EXPERIMENT_DIR,DATA_PATH,LOG_DIR)
configuration_dict

{'date_of_creation': '2025-07-17 14:54:27',
 'experiment_name': 'ordinal_wine_example',
 'PATHS': {'DATA_PATH': '/home/bule/TramDag/dev_experiment_logs/ordinal_wine_example',
  'LOG_DIR': '/home/bule/TramDag/dev_experiment_logs',
  'EXPERIMENT_DIR': '/home/bule/TramDag/dev_experiment_logs/ordinal_wine_example'},
 'data_type': None,
 'adj_matrix': None,
 'model_names': None,
 'seed': None,
 'nodes': None}

modeloling it as a graph with one sink node 

In [9]:
data_type={key:value for key, value in zip(train_df.columns, ['cont']*13+['ord'])}

levels_dict={'target':len(np.unique(df['target']))}  

columns = [
    'alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium',
    'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins',
    'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline', 'target'
]

adj_matrix = np.full((len(columns), len(columns)), "0", dtype=object)

# Set last column (edges *to* 'target') as "ls", excluding self-loop
for i in range(len(columns) - 1):
    adj_matrix[i, -1] = "ls"
    
nn_names_matrix= create_nn_model_names(adj_matrix,data_type)
nn_names_matrix

array([['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0

In [ ]:
target_nodes=create_node_dict(adj_matrix, nn_names_matrix, data_type, min_vals, max_vals,levels_dict)
target_nodes

{'alcohol': {'Modelnr': 0,
  'data_type': 'cont',
  'node_type': 'source',
  'parents': [],
  'parents_datatype': {},
  'transformation_terms_in_h()': {},
  'min': 11.665000000000001,
  'max': 14.2295,
  'transformation_term_nn_models_in_h()': {}},
 'malic_acid': {'Modelnr': 1,
  'data_type': 'cont',
  'node_type': 'source',
  'parents': [],
  'parents_datatype': {},
  'transformation_terms_in_h()': {},
  'min': 1.0710000000000002,
  'max': 4.600999999999998,
  'transformation_term_nn_models_in_h()': {}},
 'ash': {'Modelnr': 2,
  'data_type': 'cont',
  'node_type': 'source',
  'parents': [],
  'parents_datatype': {},
  'transformation_terms_in_h()': {},
  'min': 1.92,
  'max': 2.7495,
  'transformation_term_nn_models_in_h()': {}},
 'alcalinity_of_ash': {'Modelnr': 3,
  'data_type': 'cont',
  'node_type': 'source',
  'parents': [],
  'parents_datatype': {},
  'transformation_terms_in_h()': {},
  'min': 14.030000000000001,
  'max': 25.0,
  'transformation_term_nn_models_in_h()': {}},
 'm

In [ ]:
node='target'
tram_model=get_fully_specified_tram_model('target', target_nodes, verbose=True)
tram_model

Constructed TRAM model: TramModel(
  (nn_int): SimpleIntercept(
    (fc): Linear(in_features=1, out_features=2, bias=False)
  )
  (nn_shift): ModuleList(
    (0-12): 13 x LinearShift(
      (fc): Linear(in_features=1, out_features=1, bias=False)
    )
  )
)


TramModel(
  (nn_int): SimpleIntercept(
    (fc): Linear(in_features=1, out_features=2, bias=False)
  )
  (nn_shift): ModuleList(
    (0-12): 13 x LinearShift(
      (fc): Linear(in_features=1, out_features=1, bias=False)
    )
  )
)

In [ ]:
# from utils.tram_data import get_dataloader

learning_rate=0.1
use_scheduler=False


NODE_DIR = os.path.join(EXPERIMENT_DIR, f'{node}')
os.makedirs(NODE_DIR, exist_ok=True)

MODEL_PATH,LAST_MODEL_PATH,TRAIN_HIST_PATH,VAL_HIST_PATH=model_train_val_paths(NODE_DIR)


optimizer =torch.optim.Adam(tram_model.parameters(), lr=learning_rate)

if use_scheduler:
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
else:
    scheduler = None

train_loader, val_loader = get_dataloader(node, target_nodes, train_df, val_df, batch_size=128, verbose=True)


Parents dtype: OrderedDict([('alcohol', 'cont'), ('malic_acid', 'cont'), ('ash', 'cont'), ('alcalinity_of_ash', 'cont'), ('magnesium', 'cont'), ('total_phenols', 'cont'), ('flavanoids', 'cont'), ('nonflavanoid_phenols', 'cont'), ('proanthocyanins', 'cont'), ('color_intensity', 'cont'), ('hue', 'cont'), ('od280/od315_of_diluted_wines', 'cont'), ('proline', 'cont')])


In [ ]:
epochs =1000
train_val_loop(
            node,
            target_nodes,
            NODE_DIR,
            tram_model,
            train_loader,
            val_loader,
            epochs,
            optimizer,
            use_scheduler=False,
            scheduler=False,
            save_linear_shifts=False,
            verbose=1,
            device=device)


Min-Max values for target: tensor([0., 2.], device='cuda:0')
torch.Size([2])
Existing model found. Loading weights and history...


# Evaluate

In [14]:
## laoding the best model 
MODEL_PATH,LAST_MODEL_PATH,TRAIN_HIST_PATH,VAL_HIST_PATH=model_train_val_paths(NODE_DIR)

if os.path.exists(MODEL_PATH) and os.path.exists(TRAIN_HIST_PATH) and os.path.exists(VAL_HIST_PATH):
    print("Existing model found. Loading weights and history...")
    tram_model.load_state_dict(torch.load(MODEL_PATH))

Existing model found. Loading weights and history...


In [ ]:
## evaluate on testdata
tram_model.eval()

_, ordered_transformation_terms_in_h, _=ordered_parents(node, target_nodes)

min_vals = torch.tensor(target_nodes[node]['min'], dtype=torch.float32).to(device)
max_vals = torch.tensor(target_nodes[node]['max'], dtype=torch.float32).to(device)
min_max = torch.stack([min_vals, max_vals], dim=0)

with torch.no_grad():
    
    
    for x, y in val_loader:
        y = y.to(device)
        
        int_input, shift_list = preprocess_inputs(x, ordered_transformation_terms_in_h.values(), device=device)
        y_pred = tram_model(int_input=int_input, shift_input=shift_list)
        # loss = contram_nll(y_pred, y, min_max=min_max)
        pred_labels = get_pdf_ordinal(get_cdf_ordinal(y_pred)).argmax(dim=1)
        true_labels = y.argmax(dim=1)
        accuracy = (pred_labels == true_labels).float().mean().item()

        print(f"Accuracy: {accuracy*100:.1f}%")  # → 100.0%

Accuracy: 100.0%
